In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import *
import mlflow

# Configure Unity Catalog integration
mlflow.set_registry_uri("databricks-uc")
# Get current user's username dynamically
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

dbutils.widgets.text("experiment_name", "N/A")
experiment_name = dbutils.widgets.get("experiment_name")
if not experiment_name or experiment_name == "N/A":
    experiment_name = f"/Users/{username}/ml/experiments/healthcare-mlops-dev"
mlflow.set_experiment(experiment_name)

In [ ]:
with mlflow.start_run(run_name="healthcare_insurance_eda"):
    # Load data from new healthcare tables with proper joins
    # Base data from silver_patients table (replaces juan_dev.ml.insurance_silver)
    silver_patients = spark.table("juan_dev.healthcare_data.silver_patients")
    
    # Join with dimensional tables for enriched features
    # silver_patients already has most ML features, just add additional attributes from dim
    
    dim_patients = spark.table("juan_dev.healthcare_data.dim_patients").filter(col("is_current_record") == True)
    
    # Create enriched dataset with demographic attributes
    # Select only non-overlapping columns from dim_patients to avoid ambiguity
    enriched_data = (
        silver_patients
        .join(
            dim_patients.select(
                col("patient_natural_key"),
                col("health_risk_score"),
                col("lifestyle_risk_factors")
            ),
            silver_patients.patient_id == dim_patients.patient_natural_key,
            "left"
        )
        .select(
            silver_patients.patient_id,
            silver_patients.age,
            silver_patients.sex,
            silver_patients.bmi,
            silver_patients.children,
            silver_patients.smoker,
            silver_patients.region,
            silver_patients.charges,
            silver_patients.patient_age_category,
            silver_patients.health_risk_category,
            silver_patients.demographic_segment,
            silver_patients.patient_data_quality_score,
            col("health_risk_score"),
            col("lifestyle_risk_factors")
        )
    )
    
    # Convert to Pandas for EDA
    df = enriched_data.toPandas()
    
    # Healthcare-specific data profiling
    eda_results = {
        "total_patients": len(df),
        "avg_age": df['age'].mean(),
        "smoker_percentage": (df['smoker'].sum() / len(df)) * 100,
        "high_cost_threshold": df['charges'].quantile(0.95),
        "missing_data_percentage": (df.isnull().sum() / len(df)) * 100
    }
    
    # Log healthcare compliance metrics
    # mlflow.log_metrics(eda_results)
    # Log healthcare compliance metrics
    # mlflow.log_metrics({k: float(v) for k, v in eda_results.items()})
    
    # Risk factor analysis using new column mapping
    risk_analysis = df.groupby(['smoker', 'patient_age_category']).agg({
        'charges': ['mean', 'median', 'std'],
        'bmi': 'mean'
    }).round(2)
    
    # Log visualizations
    plt.figure(figsize=(12, 8))
    sns.boxplot(data=df, x='region', y='charges', hue='smoker')
    plt.title('Healthcare Costs by Region and Smoking Status')
    plt.xticks(rotation=45)
    mlflow.log_figure(plt.gcf(), "cost_distribution_by_region_smoking.png")
    
    # Feature correlation analysis
    correlation_matrix = df[['age', 'bmi', 'children', 'charges']].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Healthcare Feature Correlations')
    mlflow.log_figure(plt.gcf(), "feature_correlations.png")
    
    # Healthcare risk insights
    high_risk_patients = df[
        (df['smoker'] == True) & 
        (df['bmi'] > 30) & 
        (df['age'] > 50)
    ]
    
    mlflow.log_metrics({
        "high_risk_patients_count": len(high_risk_patients),
        "high_risk_avg_cost": high_risk_patients['charges'].mean()
    })

In [ ]:
from databricks.feature_engineering import FeatureEngineeringClient
from pyspark.sql import Window
from pyspark.sql.functions import col, expr, coalesce

fe = FeatureEngineeringClient()

# Healthcare insurance feature engineering with new schema
def create_healthcare_features():
    # Load data from new healthcare schema - filter for current records
    df = spark.table("juan_dev.healthcare_data.dim_patients").filter(col("is_current_record") == True)
    
    # Map categorical fields back to numeric for ML compatibility
    healthcare_features = (
        df
        # Convert age category to numeric age (approximate middle values)
        .withColumn("age",
                   expr("CASE WHEN patient_age_category = 'YOUNG' THEN 25 " +
                        "WHEN patient_age_category = 'ADULT' THEN 35 " +
                        "WHEN patient_age_category = 'MIDDLE_AGED' THEN 45 " +
                        "WHEN patient_age_category = 'SENIOR' THEN 60 " +
                        "ELSE 70 END"))
        
        # BMI is already numeric in dim_patients, use it directly
        # (no need to convert from category)
        
        # Use number_of_dependents directly as children
        .withColumn("children", col("number_of_dependents"))
        
        # Convert smoking status to boolean
        .withColumn("smoker", col("patient_smoking_status") == "SMOKER")
        
        # Use region and gender directly
        .withColumn("region", col("patient_region"))
        .withColumn("sex", col("patient_gender"))
        
        # Age risk scoring (recreated from original logic)
        .withColumn("age_risk_score",
                   expr("CASE WHEN age < 25 THEN 1 " +
                        "WHEN age < 35 THEN 2 " +
                        "WHEN age < 50 THEN 3 " +
                        "WHEN age < 65 THEN 4 " +
                        "ELSE 5 END"))
        
        # Smoking impact factor (recreated)
        .withColumn("smoking_impact", 
                   expr("CASE WHEN smoker THEN age * 2.5 ELSE age * 1.0 END"))
        
        # Family size risk adjustment (recreated)
        .withColumn("family_size_factor", 
                   expr("1 + (children * 0.15)"))
        
        # Regional cost adjustment (recreated)
        .withColumn("regional_multiplier",
                   expr("CASE WHEN region = 'NORTHEAST' THEN 1.2 " +
                        "WHEN region = 'NORTHWEST' THEN 1.1 " +
                        "WHEN region = 'SOUTHEAST' THEN 1.0 " +
                        "ELSE 0.95 END"))
        
        # Use existing health risk composite score or create new one
        .withColumn("health_risk_composite",
                   coalesce(col("health_risk_score"), 
                           expr("(age_risk_score * 20) + " +
                                "(CASE WHEN smoker THEN 50 ELSE 0 END) + " +
                                "(CASE WHEN bmi > 30 THEN 30 ELSE 0 END)")))
        
        # Add new features from healthcare schema
        .withColumn("data_quality_score", col("patient_data_quality_score"))
        .withColumn("hipaa_compliant", col("hipaa_deidentification_applied"))
        
        # Use patient_natural_key as the primary key for features
        .withColumn("customer_id", col("patient_natural_key"))
    )
    
    # Ensure unique customer_id
    healthcare_features = healthcare_features.dropDuplicates(["customer_id"])
    
    return healthcare_features

# Create or update feature table in Unity Catalog
healthcare_features_df = create_healthcare_features()

feature_table_name = "juan_dev.healthcare_data.ml_insurance_features"

# Check if table exists and use appropriate method
try:
    # Try to get table metadata - if this succeeds, table exists
    spark.table(feature_table_name)
    table_exists = True
except Exception as e:
    # Table doesn't exist or not accessible
    table_exists = False

if table_exists:
    # Table exists - update it with new data
    # For true overwrite behavior: delete all rows first, then merge
    print(f"Feature table {feature_table_name} already exists. Updating with new data...")
    
    # Delete all existing rows to achieve overwrite behavior
    spark.sql(f"DELETE FROM {feature_table_name}")
    print(f"Cleared existing data from {feature_table_name}")
    
    # Use merge mode to insert new data (merge is the only supported mode)
    fe.write_table(
        name=feature_table_name,
        df=healthcare_features_df,
        mode="merge"
    )
    print(f"✅ Feature table {feature_table_name} updated successfully!")
else:
    # Table doesn't exist - create it
    print(f"Creating new feature table {feature_table_name}...")
    fe.create_table(
        name=feature_table_name,
        primary_keys=["customer_id"],
        df=healthcare_features_df,
        description="Healthcare-specific features for insurance risk prediction using new schema"
    )
    print(f"✅ Feature table {feature_table_name} created successfully!")